In [1]:
import matplotlib.pyplot as plt
from ekpy import control
from ekpy.control.instruments import keysight81150a, keysightdsox3024a
from PVloop import FEHysteresis
import numpy as np

2


In [2]:
rm = control.ResourceManager()
rm.list_resources()

('USB0::0x0957::0x17A6::MY63080078::INSTR', 'GPIB0::8::INSTR')

In [3]:
scope = rm.open_resource('USB0::0x0957::0x17A6::MY63080078::INSTR')
wavegen = rm.open_resource('GPIB0::8::INSTR')

In [4]:
keysightdsox3024a.idn(scope)

'AGILENT TECHNOLOGIES,DSO-X 3024A,MY63080078,02.65.2021030741\n'

In [5]:
keysight81150a.idn(wavegen)

'Agilent Technologies,81150A,MY53821602,3.0.0.0-4.6\n'

In [8]:
exp_fe = FEHysteresis(wavegen,scope)

# specify run function (if desired)
# options are: run_function aka i havent added a different one


# config data path
exp_fe.config_path('./newsampleworksradiant')

In [9]:
voltages = ['1']
scan_params = {
    'voltage':voltages
}

run_function_args = {
    'wavegen':wavegen,
    'scope':scope,
    'initial_delay':'5e-5',
    'pulse_delay':'5e-6',
    'freq': '10',
    'voltage':'2',
    'capacitor_area':'4e-8',
    'thickness':'10e-9',
    'permittivity':'30',
	'amplification':'8',
}

order = ['voltage',]

In [11]:
exp_fe.n_param_scan(scan_params, run_function_args, order, ntrials=5, plot=False)

done.
